Add project root to path

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

Load raw data

Create dataset objects

In [2]:
import torch
from utils import filename
from utils import filehandling

size = 'dev_s'
cube_side = 32
transform = 'power'
prob = 50

directory = filename.processed.dataset(size, cube_side, transform, prob)
dataset = filehandling.read_splitted_dataset(directory)

  0%|          | 0/39 [00:00<?, ?it/s]

Create dataloader, split to train & test

In [3]:
from utils.data import splitting

train, test = splitting.train_val_split(dataset, .8)

Load pretrained 2D model

In [4]:
import segmentation_models_pytorch as smp

model = smp.Unet(encoder_name='resnet101', in_channels=1, classes=1)

Convert pretrained 2D model to 3D

In [5]:
import torch
from models.convert2Dto3D import Conv3dConverter

Conv3dConverter(model, -1, torch.ones(1, 1, 32, 32, 32))

Conv3dConverter(
Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace

In [6]:
from torch import nn
def bce_loss(y_hat, y):
    yf = y.flatten()
    y_hatf = y_hat.flatten()
    return nn.BCEWithLogitsLoss()(y_hatf, yf)

In [7]:
import numpy as np
seg = train.get_attribute('segmentmap')
threshold = 0.5#sum(map(torch.sum, seg)) / sum(map(lambda t: torch.prod(torch.tensor(t.shape)),seg))

In [8]:
from utils.data.generating import get_hi_shape
hi_shape = get_hi_shape(filename.data.sky(size))

Create Lightning objects

In [9]:
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from datetime import datetime
from pytorch_toolbelt import losses

from models.segmentation import Segmenter, get_vis_id

vis_id = get_vis_id(test, hi_shape, 50)

def loss(yh, y):
    return bce_loss(yh, y) + losses.JaccardLoss(mode='binary', log_loss=True)(yh, y)

version = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
logger = TensorBoardLogger("tb_logs", name="segmenter", version=version)
segmenter = Segmenter(model, loss, train, test, logger, vis_id=vis_id, threshold=threshold)

trainer = pl.Trainer(max_epochs=500, gpus=1, logger=logger)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train!

In [ ]:
trainer.fit(segmenter)

Set SLURM handle signals.

  | Name  | Type | Params
-------------------------------
0 | model | Unet | 112 M 
-------------------------------
112 M     Trainable params
0         Non-trainable params
112 M     Total params
/users/hhkansso/env/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/users/hhkansso/env/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/users/hhkansso/env/lib/python3.8/site-packages/pytorch_lightning/core/step_result.py:145: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)


tensor(72., device='cuda:0') tensor(165033, device='cuda:0') tensor(10170., device='cuda:0')
tensor(5., device='cuda:0') tensor(166221, device='cuda:0') tensor(6802., device='cuda:0')


/users/hhkansso/env/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


...

tensor(516., device='cuda:0') tensor(2069, device='cuda:0') tensor(5570., device='cuda:0')
tensor(2191., device='cuda:0') tensor(3856, device='cuda:0') tensor(8640., device='cuda:0')
tensor(1271., device='cuda:0') tensor(4249, device='cuda:0') tensor(7137., device='cuda:0')
tensor(3263., device='cuda:0') tensor(5996, device='cuda:0') tensor(8950., device='cuda:0')


Validating: 0it [00:00, ?it/s]

tensor(1115., device='cuda:0') tensor(1988, device='cuda:0') tensor(7440., device='cuda:0')
tensor(1793., device='cuda:0') tensor(3706, device='cuda:0') tensor(8862., device='cuda:0')
tensor(2257., device='cuda:0') tensor(4551, device='cuda:0') tensor(8585., device='cuda:0')
tensor(221., device='cuda:0') tensor(599, device='cuda:0') tensor(5410., device='cuda:0')


Validating: 0it [00:00, ?it/s]

tensor(4414., device='cuda:0') tensor(15821, device='cuda:0') tensor(10147., device='cuda:0')
tensor(3615., device='cuda:0') tensor(14929, device='cuda:0') tensor(8669., device='cuda:0')
tensor(2136., device='cuda:0') tensor(13296, device='cuda:0') tensor(5561., device='cuda:0')
tensor(1956., device='cuda:0') tensor(7995, device='cuda:0') tensor(5920., device='cuda:0')
